# Facility

Simple facility location model: given a set of plants and a set of warehouses, with transportation costs between them, this example finds the least expensive set of plants to open in order to satisfy product demand. This example demonstrates the use of MIP starts — the example computes an initial, heuristic solution and passes that solution to the MIP solver.

**Problem**  
A company currently ships products from 5 plants to 4 warehouses. The company is considering the option of closing down one or more plants. This would increase distribution cost but perhaps lower overall cost. What plants, if any, should the company close?	 


**Solution**  
1) The variables are the decisions to **open or close the plants**, and **the number of products** that should be shipped from the plants that are open to the warehouses. In worksheet Facility these are given the names `Open_or_close` and `Products_shipped`.	 
2) The logical constraints are
> `Products_shipped` >= 0 via the Assume Non-Negative option	 	 
> `Open_or_close` = binary	 

The products made can not exceed the capacity of the plants and the number shipped should meet the demand. This gives
> `Products_made` <= Capacity  
> `Total_shipped` >= Demand  

3) The objective is to minimize cost. This is given the name `Total_cost` on the worksheet.

**Remarks**  
- It is often possible to increase the capacity of a plant. This could be worked into the model with additional 0-1 or binary integer variables. The Solver would find out **if it would be profitable to extend the capacity of a plant**.  
- It could also be interesting to see if it would be profitable to **open another warehouse**. An example of this can be found, in somewhat modified form, in the capacity planning model in the Finance Examples workbook.

In [35]:
# Facility location: a company currently ships its product from 5 plants
# to 4 warehouses. It is considering closing some plants to reduce
# costs. What plant(s) should the company close, in order to minimize
# transportation and fixed costs?
#
# Note that this example uses lists instead of dictionaries.  Since
# it does not work with sparse data, lists are a reasonable option.
#
# Based on an example from Frontline Systems:
#   http://www.solver.com/disfacility.htm
# Used with permission.

import gurobipy as gp
from gurobipy import GRB

In [36]:
# Plant capacity in thousands of units
capacity = [20, 22, 17, 19, 18]

# Warehouse demand in thousands of units
demand = [15, 18, 14, 20]

# Fixed costs for each plant
fixedCosts = [12000, 15000, 17000, 13000, 16000]

# Transportation costs per thousand units
transCosts = [[4000, 2000, 3000, 2500, 4500],
              [2500, 2600, 3400, 3000, 4000],
              [1200, 1800, 2600, 4100, 3000],
              [2200, 2600, 3100, 3700, 3200]]

# Range of plants and warehouses
plants = range(len(capacity))
warehouses = range(len(demand))

In [37]:
# Model
m = gp.Model("facility")

In [55]:
# Plant open decision variables: open[p] == 1 if plant p is open.
open_ = m.addVars(plants, vtype=GRB.BINARY, obj=fixedCosts, name='open')
open_

{0: <gurobi.Var *Awaiting Model Update*>,
 1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>,
 4: <gurobi.Var *Awaiting Model Update*>}

In [39]:
# Transportation decision variables: transport[w,p] captures the
# optimal quantity to transport to warehouse w from plant p
transport = m.addVars(warehouses, plants, obj=transCosts, name='trans')

In [40]:
# You could use Python looping constructs and m.addVar() to create
# these decision variables instead.  The following would be equivalent
# to the preceding two statements...
#
# open = []
# for p in plants:
#     open.append(m.addVar(vtype=GRB.BINARY,
#                          obj=fixedCosts[p],
#                          name="open[%d]" % p))
#
# transport = []
# for w in warehouses:
#     transport.append([])
#     for p in plants:
#         transport[w].append(m.addVar(obj=transCosts[w][p],
#                                      name="trans[%d,%d]" % (w, p)))

In [41]:
# The objective is to minimize the total fixed and variable costs
m.modelSense = GRB.MAXIMIZE

In [42]:
# Production constraints
# Note that the right-hand limit sets the production to zero if the plant
# is closed

m.addConstrs((transport.sum('*', p) <= capacity[p] * open[p] for p in plants),
             name='Capacity'
             )

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>}

In [43]:
# Demand constraints
m.addConstrs((transport.sum(w, '*') == demand[w] for w in warehouses),
             name='Demand'
            )

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

In [44]:
# Save model
m.write('facilityPY.lp')

In [57]:
# Guess at the starting point: close the plant with the highest fixed costs;
# open all others
# First open all plants
for p in plants:
    open_[p].start = 1.0
open_

{0: <gurobi.Var *Awaiting Model Update*>,
 1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>,
 4: <gurobi.Var *Awaiting Model Update*>}

In [58]:
# Now close the plant with the highest fixed cost
print('Initial guess:')
maxFixed = max(fixedCosts)
p_idx = fixedCosts.index(maxFixed)
open_[p_idx].start = 0.0
print('close plant %s' % p)

Initial guess:
close plant 4


In [60]:
# Use barrier to solve root relaxation
m.Params.method = 2

Parameter method unchanged
   Value: 2  Min: -1  Max: 5  Default: -1


In [61]:
# Solve
m.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 9 rows, 40 columns and 45 nonzeros
Model fingerprint: 0x91483108
Variable types: 20 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+03, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 2e+01]

User MIP start produced solution with objective 504900 (0.02s)
Loaded user MIP start with objective 504900
MIP start from previous solve produced solution with objective 546400 (0.02s)
Loaded MIP start from previous solve with objective 546400

Presolve removed 0 rows and 20 columns
Presolve time: 0.00s
Presolved: 9 rows, 20 columns, 40 nonzeros
Variable types: 20 continuous, 0 integer (0 binary)
Root barrier log...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.000e+01
 Factor NZ  : 4.500e+01
 Factor Ops : 2.850e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter

In [62]:
# Print solution
print('\nTOTAL COSTS: %g' % m.objVal)
print('SOLUTION:')
for p in plants:
    if open[p].x > 0.99:
        print('Plant %s open' % p)
        for w in warehouses:
            if transport[w, p].x > 0:
                print('  Transport %g units to warehouse %s' %
                      (transport[w, p].x, w))
    else:
        print('Plant %s closed!' % p)


TOTAL COSTS: 546400
SOLUTION:
Plant 0 open
  Transport 15 units to warehouse 0
Plant 1 open
Plant 2 open
  Transport 15 units to warehouse 3
Plant 3 open
  Transport 14 units to warehouse 2
  Transport 5 units to warehouse 3
Plant 4 open
  Transport 18 units to warehouse 1


In [72]:
open

{0: <gurobi.Var open[0] (value 1.0)>,
 1: <gurobi.Var open[1] (value 1.0)>,
 2: <gurobi.Var open[2] (value 1.0)>,
 3: <gurobi.Var open[3] (value 1.0)>,
 4: <gurobi.Var open[4] (value 1.0)>}

In [73]:
transport

{(0, 0): <gurobi.Var trans[0,0] (value 15.0)>,
 (0, 1): <gurobi.Var trans[0,1] (value 0.0)>,
 (0, 2): <gurobi.Var trans[0,2] (value 0.0)>,
 (0, 3): <gurobi.Var trans[0,3] (value 0.0)>,
 (0, 4): <gurobi.Var trans[0,4] (value 0.0)>,
 (1, 0): <gurobi.Var trans[1,0] (value 0.0)>,
 (1, 1): <gurobi.Var trans[1,1] (value 0.0)>,
 (1, 2): <gurobi.Var trans[1,2] (value 0.0)>,
 (1, 3): <gurobi.Var trans[1,3] (value 0.0)>,
 (1, 4): <gurobi.Var trans[1,4] (value 18.0)>,
 (2, 0): <gurobi.Var trans[2,0] (value 0.0)>,
 (2, 1): <gurobi.Var trans[2,1] (value 0.0)>,
 (2, 2): <gurobi.Var trans[2,2] (value 0.0)>,
 (2, 3): <gurobi.Var trans[2,3] (value 14.0)>,
 (2, 4): <gurobi.Var trans[2,4] (value 0.0)>,
 (3, 0): <gurobi.Var trans[3,0] (value 0.0)>,
 (3, 1): <gurobi.Var trans[3,1] (value 0.0)>,
 (3, 2): <gurobi.Var trans[3,2] (value 15.0)>,
 (3, 3): <gurobi.Var trans[3,3] (value 5.0)>,
 (3, 4): <gurobi.Var trans[3,4] (value 0.0)>}